# Introduction: Simulating Asteroid Impacts with a Poisson Process

In this notebook, we will simulate impacts of near Earth asteroids into Earth using a Poisson process. Through this method, we can come up with a likely number of impacts during an individual's life and figure out how long it will be until the Earth is hit by a massive asteroid! 

In [1]:
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 20
pd.options.display.max_rows = 10

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Interactive plotting
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='pearl')

# Used for calculating theoretical value
from scipy.special import factorial

## Data on Impacts

The data on impact frequency, impact energy, and number of objects is from the NASA 2017 Report of the Near Earth Object Science Definition Team. It is available [online here](https://cneos.jpl.nasa.gov/doc/2017_neo_sdt_final_e-version.pdf).

![](images/impact_frequency.PNG)

The average impact frequency for all asteriods is $1.66 x 10^{-9} yr^{-1}$. To get the rate per size category per year, we just multiple the impact frequency times the number of asteroids in the size category. 

According to the overall frequency rate, a single Near Earth Asteroid (NEA) will impact the Earth once in 600 million years. Or, if there are 600 million NEAs, there should be one collison every year. The impact frequency in a category is the rate is the Poisson equation. To get the expected number of impacts in a time period, we multiple the rate per year times the number of years. For example, we can use 100 years as a standard human lifetime and then find the number of expected asteriods in one human lifetime. We can also use the rates to find the expected waiting time between asteriod impacts in each class.

### Acronyms

* NEA: Near Earth Asteroid
* NEO: Near Earth Object
* PHA = Potentially Hazardous Asteroids

In [2]:
import warnings; warnings.filterwarnings('ignore', category=FutureWarning)

# Read in the data
df = pd.read_parquet('data/asteroid-impact-data')
df.head()

,range_diameter,diameter,range_absolute_magnitude,absolute_magnitude,impact_energy,cumulative_number_greater,number,impact_frequency,undiscovered_fraction,undiscover_number,undiscovered_impact_frequency
0,.0200–.0251,0.0224,25.75–26.25,26.0,4.523-01,5220000.0,2850000.0,0.004730,1.000,2850000.0,0.004730
1,.0251–.0316,0.0282,25.25–25.75,25.5,9.02e-01,2370000.0,1350000.0,0.002240,1.000,1350000.0,0.002240
2,.0316–.0398,0.0355,24.75–25.25,25.0,1.80e+00,1020000.0,526000.0,0.000873,0.999,525000.0,0.000872
3,.0398–.0501,0.0447,24.25–24.75,24.5,3.59e+00,493000.0,263000.0,0.000437,0.997,262000.0,0.000435
4,.0501–.0631,0.0562,23.75–24.25,24.0,7.16e+00,230000.0,116000.0,0.000193,0.994,115000.0,0.000191


Let's make a quick plot of the impact energy by the range diameter. The diameters are given in kilometers. Impact energy is in Megaton Equivalent of TNT. For comparison, the Little Boy bomb dropped on Hiroshima had 15 kilotons of TNT equivalent. The largest human bomb ever developed was about 100 Megatons. (See [this Wikipedia article](https://en.wikipedia.org/wiki/TNT_equivalent#Examples))

In [3]:
df.set_index('range_diameter')['impact_energy'].iplot(kind='bar', xTitle='Range of Diameters (KM)', 
                                                      yTitle='Megatons Equivalent TNT', title="Impact Energy vs Diameter")

Here's the same graph with a log scale on the y axis.

In [4]:
df.set_index('range_diameter')['impact_energy'].iplot(kind='bar',
                                                      layout=dict(yaxis=dict(type='log', title='Megatons Equivalent TNT'),
                                                                 xaxis=dict(title='Range of Diameters (KM)'),
                                                      title="Impact Energy vs Diameter Log Scale"))

That is some serious firepower. We can also look at some of the other columns, such as the number in each category.

In [5]:
df.set_index('range_diameter')['number'].iplot(kind='bar',
                                                      layout=dict(yaxis=dict(type='log', title='Count'),
                                                                 xaxis=dict(title='Range of Diameters (KM)'),
                                                      title="Number of Asteroids vs Diameter Log Scale"))

# Simulations of Impacts

Here we will run 10000 simluations of a human lifetime (at 100 years). For each impact frequency, we can calculate the expected number of impacts. 

This is made simple through the `np.random.poisson` function which takes an expected value (`lambda`) and a `size` and returns the counts in each process. 

In [6]:
np.random.seed(100)

# Each simulation is a human lifetime
simulations = 10000
years = 100

def simulate_impacts(years, simulations=10000):
    # Empty array to hold simulations
    impacts = np.zeros((len(df), simulations))
    lambdas = np.zeros(len(df))

    # Iterate through each frequency
    for i, freq in enumerate(df['impact_frequency']):
        # The expected number of impacts is the frequency times the length of time
        lam = freq * years
        lambdas[i] = lam
        # Run simulations with a poisson process
        impacts[i] = np.random.poisson(lam, simulations)
        
    return impacts, lambdas

In [7]:
impacts, lambdas = simulate_impacts(years, simulations)

If we take the average value across the 10,000 simulations, we come up with an estimated number of impacts. These should be close to `lambda` for each impact frequency as `lambda` is the expected number of impacts.

In [8]:
def plot_mean_data_and_theoretical(impacts, lambdas, ds=df['range_diameter']):
    data = [go.Bar(dict(x=ds, y=impacts.mean(axis=1), name='Observed')),
            go.Bar(dict(x=ds, y=lambdas, name='Theoretical'))]
    layout = go.Layout(xaxis=dict(title='Diameter Range (km)'),
                       yaxis=dict(title='Mean Impacts'),
                      title=f'Simulated and Theoretical Asteroid Impacts in {years} Years')
    figure = go.Figure(data=data, layout=layout)
    return figure

In [9]:
figure = plot_mean_data_and_theoretical(impacts, lambdas)
iplot(figure)

## Number of Impacts

Let's inspect the results to see the average number of impacts per human lifetime in each diameter range. We will take the mean across all 10,000 simulations.

In [10]:
impact_df = pd.DataFrame(impacts.T, columns=list(df['range_diameter']))

# Plot the mean value in each column
impact_df.mean().iplot(kind='bar', xTitle='Diameter Range (km)', yTitle='Average Impacts',
                      title='Average Impacts per Human Lifetime')

There you are! You can expect an average of 0.46 impacts of the smallest asteroid in your lifetime and 0.23 of the next largest. Remember, these are still objects that can do quite some damage.

Let's look at the maximum number of expected impacts per lifetime.

In [11]:
impact_df.max().iplot(kind='bar', xTitle='Diameter Range (km)', yTitle='Maximum Impacts',
                      title='Maximum Number of Impacts per Human Lifetime')

If you are extremely unlucky, you might see five asteroids impacts of the smallest variey! 

We can also show the frequency distributions in a single category. For these, we'll show the theoretical curve derived from the Poisson Probability Mass Function (PMF because the number of events is discrete).

In [12]:
def plot_data_and_theoretical(diameter_range):
    freq = float(df.loc[df['range_diameter'] == diameter_range, 'impact_frequency'])
    lam = freq * years
    
    # Extract the data
    data = impact_df[diameter_range]
    num_events = np.arange(0, data.max() + 10, step=0.5)
    
    # Find the probability according to the Poisson PMF
    prob_num_events = 100 * np.exp(-lam) * np.power(lam, num_events) / factorial(num_events)
    
    data = [go.Scatter(x=num_events, y=prob_num_events, mode='markers+lines'), 
            go.Histogram(dict(x=data, histnorm='percent'))]
    
    layout = go.Layout(xaxis=dict(title='Number of Events'), 
                       yaxis=dict(title='Probability (%)'),
                       title=f"Observed and Theoretical Probability of Impacts for Asteroids {diameter_range} km in Diameter")
    
    figure = go.Figure(data=data, layout=layout)
    return figure

In [13]:
impact_df.columns

Index(['.0200–.0251', '.0251–.0316', '.0316–.0398', '.0398–.0501',
       '.0501–.0631', '.0631–.0794', '.0784–.1000', '.100–.126', '.126–.158',
       '.158–.200', '.200–.251', '.251–.316', '.316–.398', '.398–.501',
       '.501–.631', '.632–.794', '.794–1.00', '1.00–1.26', '1.26–1.58',
       '1.58–2.00', '2.00–2.51', '2.51–3.16', '3.16–3.98', '3.98–5.01',
       '5.01–6.31', '6.31–7.94', '7.94–10.0'],
      dtype='object')

In [14]:
figure = plot_data_and_theoretical('.0316–.0398')
iplot(figure)

In [15]:
figure = plot_data_and_theoretical('.0200–.0251')
iplot(figure)

# Increase Length of Simulation

That is all well and good, but say we are concerned with the well-being of the entire human race. Let's increase the length of time to 2 million years, about as long as the genus homo has been around. (See [this Wikipedia article](https://en.wikipedia.org/wiki/Homo))

In [16]:
np.random.seed(100)

# Each simulation is the entire history of the genus homo
simulations = 10000
years = 2000000

impacts, lambdas = simulate_impacts(years, simulations)

Let's check the mean values against the theoretical (the expected number from lambda).

In [17]:
figure = plot_mean_data_and_theoretical(impacts, lambdas)
iplot(figure)

## Mean Impacts over 2 Million Years

Let's again take a look at the mean number of impacts.

In [18]:
impact_df = pd.DataFrame(impacts.T, columns=list(df['range_diameter']))
impact_df.mean().iplot(kind='bar', xTitle='Diameter Range (km)', yTitle='Average Impacts',
                      title='Average Impacts per 2 Million Years')

Now let's see if there are any impacts of the largest asteroids in any of the simulations.

In [19]:
impact_df.max().iplot(kind='bar', xTitle='Diameter Range (km)', yTitle='Maximum Impacts',
                      title='Maximum Impacts per Million Years')

We do see at least one occasion in 10,000 simulations where the largest asteroid hits. This shows how lucky we humans are to be herewith no asteroids! 

### Simulated Data vs Theoretical 

Once again, we can plot the simluations of 2 million years versus the theoretical values.

Index(['.0200–.0251', '.0251–.0316', '.0316–.0398', '.0398–.0501',
       '.0501–.0631', '.0631–.0794', '.0784–.1000', '.100–.126', '.126–.158',
       '.158–.200', '.200–.251', '.251–.316', '.316–.398', '.398–.501',
       '.501–.631', '.632–.794', '.794–1.00', '1.00–1.26', '1.26–1.58',
       '1.58–2.00', '2.00–2.51', '2.51–3.16', '3.16–3.98', '3.98–5.01',
       '5.01–6.31', '6.31–7.94', '7.94–10.0'],

In [20]:
figure = plot_data_and_theoretical('1.58–2.00')
iplot(figure)

In [21]:
figure = plot_data_and_theoretical('2.51–3.16')
iplot(figure)

# How Long We Have to Wait for An Asteroid (Awaiting Time)

Next we'll look at the average number of years between asteroid impacts of different sizes. We'll simulate 100,000,000 (100 million) years and then find the average time between impacts.

In [22]:
# Smallest asteroids
freq = df['impact_frequency'].iloc[0]
lam = freq * years

np.random.seed(100)

# Simulate 100 million years
years = 100_000_000
wait_times = {}

# Simulate each year individually
for freq, diameter_range in zip(df['impact_frequency'], df['range_diameter']):
    a = np.random.choice([0, 1], size=1000000, p=[1-freq, freq])
    wait_times[diameter_range] = np.diff(np.where(a == 1)[0])

The wait times currently has the time between successive impacts for each size of asteroid.

In [23]:
wait_times.keys()

dict_keys(['.0200–.0251', '.0251–.0316', '.0316–.0398', '.0398–.0501', '.0501–.0631', '.0631–.0794', '.0784–.1000', '.100–.126', '.126–.158', '.158–.200', '.200–.251', '.251–.316', '.316–.398', '.398–.501', '.501–.631', '.632–.794', '.794–1.00', '1.00–1.26', '1.26–1.58', '1.58–2.00', '2.00–2.51', '2.51–3.16', '3.16–3.98', '3.98–5.01', '5.01–6.31', '6.31–7.94', '7.94–10.0'])

In [24]:
wait_times['.0200–.0251']

array([305, 136,  76, ..., 153,  42, 147], dtype=int64)

In [25]:
wait_times['6.31–7.94']

array([], dtype=int64)

There are zero impacts in 100 million years in that category.

In [26]:
wait_times['.0784–.1000']

array([ 2429, 45375,  2151, 13663,  3164, 57338,   899,  5122, 40590,
        7360, 21148,  8390, 23442,  7087, 27508,  2560, 90231,  7760,
        3071, 10200, 16038, 72854,  6976, 31092, 10950, 63976, 41691,
       50539, 12969,  2460, 12728, 14088, 24672, 58155, 18113, 16020,
       24978,   677, 14758, 15808,  1989, 25398, 29625, 33545,  8866],
      dtype=int64)

Let's take a look at plotting these wait times. We'll use a simple histogram and we can compare the simulated values to the theoretical values.

In [27]:
diameter_range = '.0251–.0316'
wt_data = wait_times[diameter_range]
freq = float(df.loc[df['range_diameter'] == diameter_range, 
                    'impact_frequency'])
sim_wt = np.arange(1, wt_data.max(), step=1)
expected = 100 * np.exp(-sim_wt * freq)

In [28]:
sim_wt

array([   1,    2,    3, ..., 3609, 3610, 3611], dtype=int64)

In [29]:
100 * expected / expected.sum()

array([2.23818021e-01, 2.23317230e-01, 2.22817559e-01, ...,
       6.91770108e-05, 6.90222278e-05, 6.88677910e-05])

In [30]:
(expected / expected.sum()).sum()

1.0

In [31]:
1 / freq

446.42857142857144

In [32]:
def plot_wait_times(diameter_range, binwidth=50):
    wt_data = wait_times[diameter_range]
    freq = float(df.loc[df['range_diameter'] == diameter_range, 
                        'impact_frequency'])
    
    sim_wt = np.arange(0, wt_data.max(), step=1)
    expected = np.exp(-sim_wt * freq)
    
    bins = np.arange(0, wt_data.max(), step=binwidth)
    
    probs = []
        
    for bin in bins:
        probs.append(1 - (expected[sim_wt <= bin].sum() / expected.sum()))
    
    probs=np.array(probs)
    probs /= probs.sum()
    
    data = [go.Histogram(dict(x=wt_data, histnorm='percent', name='Simulated', 
                              xbins=dict(start=0, end=wt_data.max(), size=binwidth))),
            go.Scatter(dict(x=bins, y=100 * probs, name='Theoretical'))]
    
    layout = go.Layout(xaxis=dict(title='Waiting Time in Years'), 
                       yaxis=dict(title='Probability (%)'),
                       title=f'Waiting Time Probabilities Observed and Theoretical for Asteroids {diameter_range} KM in Diameter')
    figure = go.Figure(data=data, layout=layout)
    return figure, probs

In [33]:
from collections import Counter
counts = Counter(wait_times['.0251–.0316'])
counts.most_common(10)

[(70, 10),
 (57, 10),
 (34, 9),
 (6, 9),
 (52, 9),
 (175, 9),
 (97, 9),
 (231, 8),
 (124, 8),
 (105, 8)]

In [34]:
figure, probs = plot_wait_times('.0251–.0316')
iplot(figure)

In [35]:
figure, probs = plot_wait_times('.0398–.0501', binwidth=200)
iplot(figure)